#  EVALUACIÓN

- Crear git.
- Crear readme.
- Fase 1. 
- Fase 2. 
- Fase 3. 

### Fase 0: Importación de librerías y configuraciones 

In [1]:
# Importamos las librerías que necesitamos
# 
# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

# Imputación de nulos usando métodos avanzados estadísticos
# -----------------------------------------------------------------------
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

# Librerías de visualización
# -----------------------------------------------------------------------
import seaborn as sns
import matplotlib.pyplot as plt

# Evaluar linealidad de las relaciones entre las variables
# ------------------------------------------------------------------------------
from scipy.stats import shapiro, kstest


# Configuraciones
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

### Fase 1: Exploración y Limpieza 

1. Exploración Inicial: 
• Realiza una exploración inicial de los datos para iden ficar posibles 
problemas, como valores nulos, a picos o datos faltantes en las columnas 
relevantes. 
• Utliza funciones de Pandas para obtener información sobre la estructura 
de los datos, la presencia de valores nulos y estadís cas básicas de las 
columnas involucradas. 
• Une los dos conjuntos de datos de la forma más eficiente. 

In [2]:
df_CFA = pd.read_csv('Customer Flight Activity.csv')

In [3]:
df_CFA.head()

,Loyalty Number,Year,Month,Flights Booked,Flights with Companions,Total Flights,Distance,Points Accumulated,Points Redeemed,Dollar Cost Points Redeemed
0,100018,2017,1,3,0,3,1521,152.0,0,0
1,100102,2017,1,10,4,14,2030,203.0,0,0
2,100140,2017,1,6,0,6,1200,120.0,0,0
3,100214,2017,1,0,0,0,0,0.0,0,0
4,100272,2017,1,0,0,0,0,0.0,0,0


In [4]:
df_CFA.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 405624 entries, 0 to 405623
Data columns (total 10 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Loyalty Number               405624 non-null  int64  
 1   Year                         405624 non-null  int64  
 2   Month                        405624 non-null  int64  
 3   Flights Booked               405624 non-null  int64  
 4   Flights with Companions      405624 non-null  int64  
 5   Total Flights                405624 non-null  int64  
 6   Distance                     405624 non-null  int64  
 7   Points Accumulated           405624 non-null  float64
 8   Points Redeemed              405624 non-null  int64  
 9   Dollar Cost Points Redeemed  405624 non-null  int64  
dtypes: float64(1), int64(9)
memory usage: 30.9 MB


In [5]:
df_CFA.duplicated().sum() 

1864

In [6]:
df_duplicados = df_CFA[df_CFA.duplicated()==True]
df_duplicados

,Loyalty Number,Year,Month,Flights Booked,Flights with Companions,Total Flights,Distance,Points Accumulated,Points Redeemed,Dollar Cost Points Redeemed
42,101902,2017,1,0,0,0,0,0.0,0,0
227,112142,2017,1,0,0,0,0,0.0,0,0
478,126100,2017,1,0,0,0,0,0.0,0,0
567,130331,2017,1,0,0,0,0,0.0,0,0
660,135421,2017,1,0,0,0,0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...
404668,949628,2018,12,0,0,0,0,0.0,0,0
404884,960050,2018,12,0,0,0,0,0.0,0,0
405111,971370,2018,12,0,0,0,0,0.0,0,0
405410,988392,2018,12,0,0,0,0,0.0,0,0


Explorando los duplicados vemos que solo tienen valores en Loyalty Number, Year y Month. 
El resto de columnas es cero. No nos aportan información.
Decido eliminar estos registros.

In [7]:
# Elimino registros duplicados

registros_duplicados = df_CFA.duplicated()[df_CFA.duplicated()==True]
filas_a_eliminar = registros_duplicados.index.to_list() # Pasamos a lista los índices de los registros duplicados sin información útil
df_CFA_sin_duplicados = df_CFA.drop(filas_a_eliminar, axis=0)
a = df_CFA.shape
b = df_CFA_sin_duplicados.shape

print(f"Hay {df_CFA_sin_duplicados.duplicated().sum()} registros duplicados")


Hay 0 registros duplicados


In [8]:
df_CFA_sin_duplicados.describe()

,Loyalty Number,Year,Month,Flights Booked,Flights with Companions,Total Flights,Distance,Points Accumulated,Points Redeemed,Dollar Cost Points Redeemed
count,403760.000000,403760.000000,403760.000000,403760.000000,403760.000000,403760.000000,403760.000000,403760.000000,403760.000000,403760.000000
mean,549875.383713,2017.500352,6.501335,4.134050,1.036569,5.170619,1214.460979,124.263761,30.838587,2.495973
std,258961.514684,0.500000,3.451982,5.230064,2.080472,6.526858,1434.098521,146.696179,125.758002,10.172033
min,100018.000000,2017.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,326699.000000,2017.000000,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,550598.000000,2018.000000,7.000000,1.000000,0.000000,1.000000,525.000000,53.000000,0.000000,0.000000
75%,772152.000000,2018.000000,10.000000,8.000000,1.000000,10.000000,2342.000000,240.000000,0.000000,0.000000
max,999986.000000,2018.000000,12.000000,21.000000,11.000000,32.000000,6293.000000,676.500000,876.000000,71.000000


Interpretación

In [9]:
# OBSERVACIONES: 

# Filas duplicadas. Eliminar ✔
# No hay valores nulos. ✔ 
# Points Accumulated estaría mejor en tipo int ✔
# Loyalty Number, Year y Year estarían mejor como object (categóricas) y no como número ✔

In [10]:
# Cambio tipos de columnas.

df_CFA_sin_duplicados['Loyalty Number'] = df_CFA_sin_duplicados['Loyalty Number'].astype('object')
df_CFA_sin_duplicados['Year'] = df_CFA_sin_duplicados['Year'].astype('object')
df_CFA_sin_duplicados['Month'] = df_CFA_sin_duplicados['Month'].astype('object')
df_CFA_sin_duplicados['Points Accumulate'] = df_CFA_sin_duplicados['Points Accumulated'].astype('int')

In [11]:
df_CFA_sin_duplicados.info()

<class 'pandas.core.frame.DataFrame'>
Index: 403760 entries, 0 to 405623
Data columns (total 11 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Loyalty Number               403760 non-null  object 
 1   Year                         403760 non-null  object 
 2   Month                        403760 non-null  object 
 3   Flights Booked               403760 non-null  int64  
 4   Flights with Companions      403760 non-null  int64  
 5   Total Flights                403760 non-null  int64  
 6   Distance                     403760 non-null  int64  
 7   Points Accumulated           403760 non-null  float64
 8   Points Redeemed              403760 non-null  int64  
 9   Dollar Cost Points Redeemed  403760 non-null  int64  
 10  Points Accumulate            403760 non-null  int64  
dtypes: float64(1), int64(7), object(3)
memory usage: 37.0+ MB


In [13]:
# Ahora vamos a crear una nueva columna con un identificador único para los datos por cliente y fecha
# porque la actividad se mide por identificador, mes y año.

def crear_identificador_unico (id_cliente, mes, año):
    id = ""
    id = str(id_cliente) + '_' + str(mes) + '_'  + str(año)
    return id


# aplicamos la función al DataFrame usando un apply y un lambda

df_CFA_sin_duplicados['id_year_month'] = df_CFA_sin_duplicados.apply(lambda x: crear_identificador_unico(x['Loyalty Number'], x['Month'], x['Year']), axis=1)

df_CFA_sin_duplicados['id_year_month'].head()


0    100018_1_2017
1    100102_1_2017
2    100140_1_2017
3    100214_1_2017
4    100272_1_2017
Name: id_year_month, dtype: object

Tras eliminar los registros duplicados, veo que sigue habiendo registros para los que se repite información y no debería. Es decir, aunque ya no haya duplicados, identifico que hay más de una anotación para para un mismo cliente, un mismo año y un mismo mes. Esto no debería ser así.

In [49]:
identificadores_unicos = list(df_CFA_sin_duplicados['id_year_month'].values)
print(f"En total debería haber {len(identificadores_unicos)} identificadores_unicos...")
set_identificadores_unicos = set(identificadores_unicos)
print(f"Pero que no se repitan solo hay {len(set_identificadores_unicos)}.")
print(f"Hay {len(identificadores_unicos) - len(set_identificadores_unicos)} registros para los que se ha recopilado información en más de un sitio")

En total debería haber 403760 identificadores_unicos...
Pero que no se repitan solo hay 401688.
Hay 2072 registros para los que se ha recopilado información en más de un sitio


In [50]:
identificadores_unicos # Lista de identificadores 

# Conjunto para almacenar valores únicos
vistos = set()
# Lista para almacenar valores repetidos
repetidos = set()

for identificador in identificadores_unicos:
    if identificador in vistos:
        repetidos.add(identificador)
    else:
        vistos.add(identificador)

print(repetidos)

{'593252_7_2018', '979115_11_2018', '902608_12_2018', '336882_7_2017', '336882_10_2017', '351615_12_2018', '255475_6_2018', '615439_6_2018', '825593_11_2017', '992168_11_2017', '700650_7_2017', '336882_3_2017', '874780_2_2018', '328623_6_2018', '617489_6_2018', '470998_10_2018', '988392_1_2017', '200519_10_2018', '112142_10_2017', '323821_3_2017', '615561_3_2017', '927026_11_2018', '112142_4_2018', '890702_7_2017', '547522_7_2018', '876062_1_2017', '402697_8_2018', '682979_8_2018', '965605_4_2017', '876062_5_2017', '181082_1_2017', '890702_1_2017', '200519_3_2017', '650466_12_2017', '833793_2_2018', '890361_11_2018', '969318_5_2017', '200519_10_2017', '650466_3_2017', '322169_11_2017', '193662_2_2018', '903359_1_2018', '958491_11_2017', '682979_7_2017', '255836_5_2018', '992168_10_2017', '584833_11_2018', '135421_6_2017', '114414_1_2018', '927026_7_2018', '198499_6_2017', '411523_2_2017', '278854_3_2017', '547522_2_2018', '899760_11_2018', '678205_8_2018', '684881_1_2017', '981508_4_20

Teniendo en cuenta que la proporción de registros con información inconsistente es de 2072 de más de 400 000 registros (equivale a un 0.5% de los datos), decido prescindir de estos registros porque no me parecen fiables.

Adjunto captura de pantalla en esta carpeta con la información repetida para no ensuciar el jupyter: 'registros_informacion_inconsistente.png'

Por otra parte, debido a que la proporción detectada de datos erróneos solo es del 0.5% y no me va a contaminar excesivamente los datos, sigo adelante con el ejercicio.



In [16]:
df_CFA_limpios = df_CFA_sin_duplicados.copy()

In [18]:
df_CFA_limpios.to_csv('datos_limpios_CFA.csv')